In [1]:
import os
from vanna.remote import VannaDefault
from dotenv import load_dotenv
from vanna.flask import VannaFlaskApp

In [2]:
load_dotenv()
api_key = os.getenv('VANNA_API_KEY')
vanna_model_name = os.getenv('VANNA_MODEL')
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

In [3]:
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT", "5432")  # default to 5432 if not set

vn.connect_to_postgres(
    host=db_host,
    dbname=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)


In [4]:
df_tables = vn.run_sql("""
SELECT
    table_catalog,
    table_schema,
    table_name
FROM INFORMATION_SCHEMA.TABLES
WHERE table_type = 'BASE TABLE'
  AND table_schema NOT IN ('pg_catalog', 'information_schema');
""")

for index, row in df_tables.iterrows():
    doc_line = f"Database: {row['table_catalog']} / Schema: {row['table_schema']} / Table: {row['table_name']}"
    vn.train(documentation=doc_line)

Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....


In [5]:
df_fk = vn.run_sql("""
SELECT
  tc.table_catalog,               -- add table_catalog here, too!
  tc.table_schema,
  tc.table_name,
  kcu.column_name,
  ccu.table_schema AS foreign_table_schema,
  ccu.table_name AS foreign_table_name,
  ccu.column_name AS foreign_column_name
FROM information_schema.table_constraints AS tc
JOIN information_schema.key_column_usage AS kcu
  ON tc.constraint_name = kcu.constraint_name
  AND tc.table_schema = kcu.table_schema
JOIN information_schema.constraint_column_usage AS ccu
  ON ccu.constraint_name = tc.constraint_name
  AND ccu.table_schema = tc.table_schema
WHERE tc.constraint_type = 'FOREIGN KEY'
  AND tc.table_schema NOT IN ('pg_catalog', 'information_schema');
""")

for index, row in df_fk.iterrows():
    doc_line = f"Database: {row['table_catalog']} / Schema: {row['table_schema']} / Table: {row['table_name']}"
    vn.train(documentation=doc_line)


Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....


In [6]:
vn.train(documentation="All the Captains, Agents, and Organization related people are in User table with different userTypes, and UserType is another Table which specify type of user. and Customers table consists of registered users, passengers table consists of passenegers who travelled with us and need not to be in customers.")
vn.train(documentation="The User table has all the information about the users, and the UserType table has the type of users. The Customers table has the information about the registered users, and the Passengers table has the information about the passengers who have traveled with us.")
vn.train(documentation="ReservedTickets table consists of all booked tickets and trip details, passenger details")
vn.train(documentation="TicketTransaction table consists of all the transactions related to tickets, and this table has all the payment details with necessary columns of what platform and when it got booked")
vn.train(documentation="ReservedTicketSeats table consists of and triplevel details, seatIds, passenger details of seat and so on")
vn.train(documentation="AgentPaymentTransaction table consists of all the transactions related to agents, and this table has all the payment details with necessary columns of what platform and when it got booked")
vn.train(documentation="Revenue should be calculated from ReservedTicketSeats table and not from TicketTransactions table")
vn.train(documentation="Blocked Seats will be in ReservedTicketSeats table with status as available = false and dummy tickets will also be in ReservedTicketSeats table with status as available = false and availableAt is not null and will be equal to currentTime of first TripboardingPoint of that trip")

Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....


'2988773-doc'

In [7]:
vn.train(question="how many dummies are there today?",sql=''' 
         SELECT count(ts."seatId") as seatcount
        FROM "Trips" t
        JOIN "TripSeats" ts ON t.id = ts."tripId"
        JOIN "TripBoardingPoints" tbp on t.id = tbp."tripId" 
        WHERE t."journeyDate" = current_date
        AND ts.available = FALSE
        AND ts."availableAt" = tbp."currentTime"
        ''')

'f25617012b3f73baec69fce550e6f9e2-sql'

In [8]:
vn.train(question="What is the Revenue in June 2024, by category wise?",sql='''
SELECT rt."mainCategory",rt."subCategory",
    SUM(rts."amount") as total_revenue
FROM "ReservedTicketSeats" rts
JOIN "ReservedTickets" rt ON rts."ticketId" = rt.id
JOIN "Trips" t ON rt."tripId" = t.id
WHERE DATE_TRUNC('month', t."journeyDate") = DATE '2024-06-01'
AND rts."active"= true
GROUP BY rt."mainCategory",rt."subCategory";
''')

'940c087f49b4060f9a48730f3f6f2e0f-sql'

In [9]:
vn.train(question="How many customers travelled with us(freshbus) in january 2025?",
         sql='''
        SELECT COUNT(DISTINCT c.id) AS total_customers
        FROM "Customers" c
        JOIN "ReservedTickets" rt ON c.id = rt."customerId"
        JOIN "Trips" t ON rt."tripId" = t.id
        WHERE DATE_TRUNC('month', t."journeyDate") = DATE '2025-01-01'
        AND rt."active"=true;
         ''')

'5ec564878d618830758f3b7ba38594f3-sql'

In [10]:
vn.train(question="What is the LookToBook Ratio for Yesterday?",
         sql='''
            SELECT 
    "subCategory",
    CASE 
        WHEN SUM(CASE WHEN "visitType" = 'Look' THEN visits ELSE 0 END) = 0 
        THEN NULL
        ELSE 
            (CAST(SUM(CASE WHEN "visitType" = 'Book' THEN visits ELSE 0 END) AS FLOAT) / 
             SUM(CASE WHEN "visitType" = 'Look' THEN visits ELSE 0 END)) * 100
    END AS look_to_book_ratio
FROM "LookToBooks"
WHERE 
    "journeyDate" = current_date - 1
    AND "visitType" IN ('Look', 'Book')
GROUP BY "subCategory";
        ''')

'c57544af0ad4979e0b39c76ee47e4986-sql'